In [94]:
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
# for getting rid of outliers
from scipy.stats.mstats import winsorize

# Setup Data Frame

Going to use next weeks fantasy points as the predictor of this weeks data

In [95]:
# creating the path to the files of this week and next week

def make_week_df(df_curr,df_next):
    df_next = df_next[['Player','PPRFantasyPoints']]
    df_next.rename(columns={'PPRFantasyPoints':'next_week_points'},inplace=True)
    fin_df_curr = df_curr.merge(df_next,how='left',on='Player')
    return fin_df_curr

def get_week_df(year,this_week):
    # next week number is this +1
    next_week = this_week+1
    # making csv path
    this_week_path = 'week'+str(this_week)+'.csv'
    next_week_path = 'week'+str(next_week)+'.csv'
    this_week_path = os.path.join('./data_v2/weekly/',str(year),str(this_week_path))
    next_week_path = os.path.join('./data_v2/weekly/',str(year),str(next_week_path))
    # opening both dataframs from csv
    df_curr = pd.read_csv(this_week_path)
    df_next = pd.read_csv(next_week_path)
    # making the df of just this week
    df = make_week_df(df_curr,df_next)
    return df

In [96]:
# initialize df
final_df = pd.DataFrame()
# iterate through years from 2010 to 2019
for year_num in range(2010,2020):
    for week_num in range(1,17):
        df_week = get_week_df(year_num,week_num)
        final_df = final_df.append(df_week)
        
df = final_df

# Dropping Unecessary Rows

In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44988 entries, 0 to 305
Data columns (total 20 columns):
Player                   44988 non-null object
Pos                      44974 non-null object
Tm                       44988 non-null object
PassingYds               44988 non-null float64
PassingTD                44988 non-null float64
Int                      44988 non-null float64
PassingAtt               44988 non-null float64
Cmp                      44988 non-null float64
RushingAtt               44988 non-null float64
RushingYds               44988 non-null float64
RushingTD                44988 non-null float64
Rec                      44988 non-null float64
Tgt                      44988 non-null float64
ReceivingYds             44988 non-null float64
ReceivingTD              44988 non-null float64
FL                       44988 non-null float64
PPRFantasyPoints         44988 non-null float64
StandardFantasyPoints    44988 non-null float64
HalfPPRFantasyPoints     44988 n

In [98]:
df.head()

,Player,Pos,Tm,PassingYds,PassingTD,Int,PassingAtt,Cmp,RushingAtt,RushingYds,RushingTD,Rec,Tgt,ReceivingYds,ReceivingTD,FL,PPRFantasyPoints,StandardFantasyPoints,HalfPPRFantasyPoints,next_week_points
0,Vince Young,QB,TEN,154.0,2.0,0.0,17.0,13.0,7.0,30.0,0.0,0.0,0.0,0.0,0.0,1.0,15.16,15.16,15.16,-2.16
1,David Garrard,QB,JAX,170.0,3.0,0.0,21.0,16.0,7.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,19.80,19.80,19.80,2.82
2,Tom Brady,QB,NWE,258.0,3.0,0.0,35.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.32,22.32,22.32,11.92
3,Peyton Manning,QB,IND,433.0,3.0,0.0,57.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29.32,29.32,29.32,21.90
4,Jay Cutler,QB,CHI,372.0,2.0,1.0,35.0,23.0,5.0,22.0,0.0,0.0,0.0,0.0,0.0,1.0,21.08,21.08,21.08,23.38


In [99]:
# dropping:
# name, the team, and points for non PPR leagues
df.drop(columns=['Player','Tm','StandardFantasyPoints','HalfPPRFantasyPoints','FL'],inplace=True)

# Going to split based on the position becasue they have much different indicators of sucess

In [100]:
# making df for the top 4 and most common positions in a fantasy league
df_qb = df[(df.Pos == 'QB')]
df_rb = df[(df.Pos == 'RB')]
df_wr = df[(df.Pos == 'WR')]
df_te = df[(df.Pos == 'TE')]

# DFA QB

In [101]:
df_qb.describe()

,PassingYds,PassingTD,Int,PassingAtt,Cmp,RushingAtt,RushingYds,RushingTD,Rec,Tgt,ReceivingYds,ReceivingTD,PPRFantasyPoints,next_week_points
count,5579.000000,5579.000000,5579.000000,5579.000000,5579.000000,5579.00000,5579.000000,5579.000000,5579.000000,5579.000000,5579.000000,5579.000000,5579.000000,4453.000000
mean,214.853916,1.324072,0.759634,29.958415,18.673597,2.76537,11.494175,0.106112,0.014877,0.016132,0.097509,0.001792,13.813730,15.032216
std,104.959530,1.185250,0.936173,13.135982,8.706034,2.51492,17.573401,0.332095,0.135070,0.152984,1.769880,0.042303,8.729149,8.167407
min,-3.000000,0.000000,0.000000,0.000000,0.000000,0.00000,-17.000000,0.000000,0.000000,0.000000,-16.000000,0.000000,-6.960000,-6.960000
25%,159.000000,0.000000,0.000000,24.000000,14.000000,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.480000,9.340000
50%,226.000000,1.000000,1.000000,32.000000,20.000000,2.00000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.880000,14.720000
75%,287.000000,2.000000,1.000000,38.000000,24.000000,4.00000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,19.580000,20.300000
max,527.000000,7.000000,6.000000,68.000000,45.000000,26.00000,152.000000,3.000000,3.000000,3.000000,63.000000,1.000000,49.320000,49.320000


In [103]:
df_qb.drop(columns = ['Rec','Tgt','ReceivingYds','ReceivingTD'],inplace=True)

In [105]:
features = df_qb.drop(columns='next_week_points')
target = df_qb['next_week_points']